In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Each user is responsible for checking the content of datasets and the
# applicable licenses and determining if suitable for the intended use.

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_models_01-getting-started/nvidia_logo.png" style="width: 90px; float: right;">

# Getting Started with Merlin Models: Develop a Model for MovieLens

This notebook is created using the latest stable [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow/tags) container. 

## Overview

[Merlin Models](https://github.com/NVIDIA-Merlin/models/) is a library for training recommender models. Merlin Models let Data Scientists and ML Engineers easily train standard RecSys models on their own dataset, getting GPU-accelerated models with best practices baked into the library. This will also let researchers to build custom models by incorporating standard components of deep learning recommender models, and then benchmark their new models on example offline datasets. Merlin Models is part of the [Merlin open source framework](https://developer.nvidia.com/nvidia-merlin).

Core features are:
- Many different recommender system architectures (tabular, two-tower, sequential) or tasks (binary, multi-class classification, multi-task)
- Flexible APIs targeted to both production and research
- Deep integration with NVIDIA Merlin platform, including NVTabular for ETL and Merlin Systems model serving


### Learning objectives

- Training [Facebook's DLRM model](https://arxiv.org/pdf/1906.00091.pdf) very easily with our high-level API.
- Understanding Merlin Models high-level API

## Downloading and preparing the dataset

In [2]:
import os
import merlin.models.tf as mm

from merlin.datasets.entertainment import get_movielens

2023-01-09 10:18:00.286093: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-09 10:18:02.454892: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-09 10:18:04.585508: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16249 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:15:00.0, compute capability: 7.0
2023-01-09 10:18:04.586621: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created

We provide the `get_movielens()` function as a convenience to download the dataset, perform simple preprocessing, and split the data into training and validation datasets.

In [3]:
input_path = os.environ.get("INPUT_DATA_DIR", os.path.expanduser("~/merlin-models-data/movielens/"))
train, valid = get_movielens(variant="ml-1m", path=input_path)

/home/gmoreira/projects/nvidia/nvidia_merlin/core/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/home/gmoreira/projects/nvidia/nvidia_merlin/core/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


## Training the DLRM Model with Merlin Models

We define the DLRM model, whose prediction task is a binary classification. From the `schema`, the categorical features are identified (and embedded) and the target column is also automatically inferred, because of the schema tags. We talk more about the schema in the next [example notebook (02)](02-Merlin-Models-and-NVTabular-integration.ipynb),

In [5]:
model = mm.DLRMModel(
    train.schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.OutputBlock(train.schema),
)

model.compile(optimizer="adam")

Next, we train the model.

In [6]:
model.fit(train, batch_size=1024)

782/782 [==============================] - 21s 22ms/step - loss: 1.5975 - rating/regression_output_loss: 1.0380 - rating_binary/binary_output_loss: 0.5596 - rating/regression_output/root_mean_squared_error: 1.0188 - rating_binary/binary_output/precision: 0.7079 - rating_binary/binary_output/recall: 0.8442 - rating_binary/binary_output/binary_accuracy: 0.7098 - rating_binary/binary_output/auc: 0.7728 - regularization_loss: 0.0000e+00 - loss_batch: 1.5970


We evaluate the model...

In [7]:
metrics = model.evaluate(valid, batch_size=1024, return_dict=True)

/home/gmoreira/projects/nvidia/nvidia_merlin/core/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/home/gmoreira/projects/nvidia/nvidia_merlin/core/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


196/196 [==============================] - 4s 14ms/step - loss: 1.3249 - rating/regression_output_loss: 0.7924 - rating_binary/binary_output_loss: 0.5326 - rating/regression_output/root_mean_squared_error: 0.8901 - rating_binary/binary_output/precision: 0.7383 - rating_binary/binary_output/recall: 0.8258 - rating_binary/binary_output/binary_accuracy: 0.7322 - rating_binary/binary_output/auc: 0.7995 - regularization_loss: 0.0000e+00 - loss_batch: 1.3255


... and check the evaluation metrics. We use by default typical binary classification metrics -- Precision, Recall, Accuracy and AUC. But you can also provide your own metrics list by setting `BinaryClassificationTask(..., metrics=[])`.

In [8]:
metrics

{'loss': 1.3249409198760986,
 'rating/regression_output_loss': 0.7923521995544434,
 'rating_binary/binary_output_loss': 0.5325889587402344,
 'rating/regression_output/root_mean_squared_error': 0.8901416659355164,
 'rating_binary/binary_output/precision': 0.7382509112358093,
 'rating_binary/binary_output/recall': 0.8258424997329712,
 'rating_binary/binary_output/binary_accuracy': 0.7322099208831787,
 'rating_binary/binary_output/auc': 0.7994654178619385,
 'regularization_loss': 0.0,
 'loss_batch': 1.3920133113861084}

## Conclusion

Merlin Models enables users to define and train a deep learning recommeder model with only 3 commands.

```python
model = mm.DLRMModel(
    train.schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.OutputBlock(
        train.schema.select_by_tag(Tags.TARGET).column_names[0]
    ),
)
model.compile(optimizer="adam")
model.fit(train, batch_size=1024)
```

## Next steps

In the next example notebooks, we will show how the integration with NVTabular and how to explore different recommender models.